# 2. Slug Test for Confined Aquifer - Multi-well

### Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import timflow.transient as tft

plt.rcParams["figure.figsize"] = [5, 3]

## Introduction and Conceptual Model

A well (Ln-2) fully penetrates a sandy confined aquifer of 6.1 m thickness. Additionally, a fully penetrating observation well (Ln-3) is located 6.45 m away from the test well. The slug displacement is 2.798 m. Head change was recorded at the slug well and the observation well. The well and casing radii of the slug well are 0.102 and 0.051 m, respectively. For the observation well, they are 0.051 and 0.025 m, respectively.

<img src="./figs/Multi_well.png" style="width:400pt">

### Load data

In [ ]:
data1 = np.loadtxt("data/ln-2.txt")
t1 = data1[:, 0] / 60 / 60 / 24  # convert time from seconds to days
h1 = data1[:, 1]

data2 = np.loadtxt("data/ln-3.txt")
t2 = data2[:, 0] / 60 / 60 / 24
h2 = data2[:, 1]

### Parameters and model

In [ ]:
# known parameters
H0 = 2.798  # initial displacement, m
b = 6.1  # aquifer thickness, m
rw1 = 0.102  # well radius of Ln-2 Well, m
rw2 = 0.071  # well radius of observation Ln-3 Well, m
rc1 = 0.051  # casing radius of Ln-2 Well, m
r = 6.45  # distance from observation well to test well, m

Converting slug displacement into volume

In [ ]:
Q = np.pi * rc1**2 * H0
print(f"Slug: {Q:.5f} m^3")

In [ ]:
ml = tft.ModelMaq(kaq=10, z=[0, -b], Saq=1e-4, tmin=1e-5, tmax=0.01)
w = tft.Well(ml, xw=0, yw=0, rw=rw1, rc=rc1, tsandQ=[(0, -Q)], layers=0, wbstype="slug")
ml.solve()

### Estimate aquifer parameters
The hydraulic conductivity and specific storage are calibrated.

In [ ]:
# unknown parameters: kaq, Saq
cal = tft.Calibrate(ml)
cal.set_parameter(name="kaq", layers=0, initial=10)
cal.set_parameter(name="Saq", layers=0, initial=1e-4)
cal.seriesinwell(name="Ln-2", element=w, t=t1, h=h1)
cal.series(name="Ln-3", x=r, y=0, layer=0, t=t2, h=h2)
cal.fit()

In [ ]:
display(cal.parameters.loc[:, ["optimal"]])
print(f"RMSE: {cal.rmse():.5f} m")

In [ ]:
hm1 = w.headinside(t1)
hm2 = ml.head(r, 0, t2, layers=0)
plt.semilogx(t1, h1 / H0, ".", label="obs ln-2")
plt.semilogx(t1, hm1[0] / H0, label="timflow ln-2")
plt.semilogx(t2, h2 / H0, ".", label="obs ln-3")
plt.semilogx(t2, hm2[0] / H0, label="timflow ln-3")
plt.xlabel("time [d]")
plt.ylabel("Normalized Head: h/H0")
plt.title("Model Results - Single layer model")
plt.legend()
plt.grid()

### Comparison of results

The values in `timflow` are compared with the KGS analytical model (Hyder et al. 1994) implemented in AQTESOLV (Duffield, 2007), and to the MLU model (Carlson & Randall, 2012). The parameters of all three models are very similar.

In [ ]:
t = pd.DataFrame(
    columns=["k [m/d]", "Ss [1/m]", "RMSE [m]"],
    index=["timflow", "AQTESOLV", "MLU"],
)

t.loc["timflow"] = np.append(cal.parameters["optimal"].values, cal.rmse())
t.loc["AQTESOLV"] = [1.166, 9.368e-06, 0.009151]
t.loc["MLU"] = [1.311, 8.197e-06, 0.010373]

t_formatted = t.style.format(
    {"k [m/d]": "{:.2f}", "Ss [1/m]": "{:.2e}", "RMSE [m]": "{:.3f}"}
)
t_formatted

### References

* Butler Jr., J.J. (1998), The Design, Performance, and Analysis of Slug Tests, Lewis Publishers, Boca Raton, Florida, 252p.
* Carlson F. and Randall J. (2012), MLU: a Windows application for the analysis of aquifer tests and the design of well fields in layered systems, Ground Water 50(4):504–510.
* Duffield, G.M. (2007), AQTESOLV for Windows Version 4.5 User's Guide, HydroSOLVE, Inc., Reston, VA.
* Hyder, Z., Butler Jr, J.J., McElwee, C.D. and Liu, W. (1994), Slug tests in partially penetrating wells, Water Resources Research 30, 2945–2957.